In [ ]:
import xml.etree.ElementTree as ET
import dateutil.parser
import math

In [ ]:
class TrackPoint:
    table_name = 'points'
    columns = [
        ('point_id', 'INTEGER PRIMARY KEY'),
        ('route_id', 'INTEGER'),
        ('datetime_text', 'TEXT'),
        ('dt', 'DATE'),
        ('lat', 'REAL'),
        ('lon', 'REAL'),
        ('ele', 'REAL')
    ]

    @classmethod
    @property
    def create_sql(cls):
        col_defs = ', '.join(' '.join(col) for col in cls.columns)
        print(col_defs)
        return f'CREATE TABLE {cls.table_name} ({col_defs})'

    def __init__(self, pt):
        self.lat = float(pt.attrib['lat'])
        self.lon = float(pt.attrib['lon'])
        self.ele = float(pt.find('{http://www.topografix.com/GPX/1/1}ele').text)
        self.date_time = pt.find('{http://www.topografix.com/GPX/1/1}time').text
        self.dt = dateutil.parser.parse(self.date_time)

    @property
    def coord(self):
        return (self.lon, self.lat)

    @classmethod
    @property
    def sql_insert_fields(cls):
        return [col[0] for col in cls.columns if not 'PRIMARY KEY' in col[1]]

    def sql_values(self, route_id):
        return (route_id, self.date_time, self.dt, self.lat, self.lon, self.ele)

    def __repr__(self):
            return f'{self.date_time}, ({self.lat}, {self.lon}), {self.ele}m'


In [ ]:
print(','.join(TrackPoint.create_sql))

In [ ]:
import haversine

class Segment:

    create_sql = '''DROP TABLE IF EXISTS segments;
        CREATE TABLE segments (
            from_point_id INTEGER,
            to_point_id INTEGER,
            elapsed_secs REAL,
            since_start REAL,
            distance REAL,
            xdistance REAL,
            delta_ele REAL
        )'''

    def __init__(self, from_pt, to_pt):
        self.from_pt = from_pt
        self.to_pt = to_pt

    @property
    def elapsed_secs(self):
        return (self.to_pt.dt - self.from_pt.dt).total_seconds()

    @property
    def delta_ele(self):
        return self.to_pt.ele - self.from_pt.ele
        
    @property
    def dist(self):
        return haversine.distance(self.to_pt.coord, self.from_pt.coord)

    @property
    def xdist(self):
        return math.sqrt((self.dist * self.dist + self.delta_ele * self.delta_ele))

    def to_sql(self):
        return ""


In [ ]:
class Route:
    create_sql = '''
        DROP TABLE IF EXISTS routes;
        CREATE TABLE routes (
            route_id INTEGER PRIMARY KEY,
            path TEXT
        );'''

    def __init__(self, path):
        self.path = path
        self.route_id = None

    def add_to_db(self, conn):
        sql = 'INSERT INTO routes (path) VALUES (?)'
        csr = conn.cursor()
        csr.execute(sql, (self.path, ))
        self.route_id = csr.lastrowid
        csr.close()
        conn.commit()
        print(f'inserted route for {self.path} as {self.route_id}')

        root = ET.parse(self.path).getroot()
        pts = [TrackPoint(pt) for pt in root.findall('.//{http://www.topografix.com/GPX/1/1}trkpt')]
        insert_values = [pt.sql_values(self.route_id) for pt in pts]
        
        sql = 'INSERT INTO points (' + ','.join(TrackPoint.sql_insert_fields) + ' VALUES (' + ', '.join(list('?' + len(TrackPoint.sql_insert_fields))) + ')'
        res = csr.executemany(sql, insert_values)
        csr.close()
        conn.commit()

In [ ]:
import itertools
import sqlite3

def pairwise(iterable):
    prev_item = None
    for current_item in iterable:
        yield (current_item, prev_item)
        prev_item = current_item

class TrackPointDbBuilder:
    def __init__(self, db_path='mwalks.sqlite'):
        self.db_path = db_path

    def create_db(self):
        self.connect()
        self.conn.executescript(Route.create_sql)
        self.conn.executescript(f'DROP TABLE IF EXISTS {TrackPoint.table_name};{TrackPoint.create_sql}')
        self.conn.executescript(Segment.create_sql)
        self.disconnect()

    def add_route_file(self, path):
        route = Route(path)
        conn = self.connect()
        route.add_to_db(conn)

    def connect(self):
        self.conn = sqlite3.connect(self.db_path)
        return self.conn

    def disconnect(self):
        self.conn.close()

In [ ]:
builder = TrackPointDbBuilder()
builder.create_db()

In [ ]:
from glob import glob

for path in glob('routes/*.gpx'):
    builder.add_route_file(path)

In [ ]:
conn = sqlite3.connect(builder.db_path)
print('database now contains:')
for tname in ['routes', 'points', 'segments']:
    c = conn.execute(f'SELECT Count(*) FROM {tname}')
    rows = c.fetchone()[0]
    print(f'    {rows} {tname}')
conn.close()